### ________________________________________________________________________________________________________________________________________
## Analysis of Taxonomy Composition over Time
#### Dataset: Microbiome of the Georiga Aquarium Ocean Voyager exhibit
#### SRA Accession PRJNA417313
### ________________________________________________________________________________________________________________________________________

### Setup

In [1]:
# Import module to output interactive plots in notebook
# Restart Jupyter after running
from qiime2 import Visualization
!jupyter serverextension enable --py qiime2 --sys-prefix

Enabling: qiime2.jupyter
- Writing config: /Users/victoriac/mambaforge/envs/qiime2-2023.2/etc/jupyter
    - Validating...
      qiime2.jupyter  OK


In [2]:
### Already run -- don't run again if paired-end-demux.qza already exists
# Import the data into QIIME
# !qiime tools import \
# --type 'SampleData[PairedEndSequencesWithQuality]' \
# --input-path ov-manifest.tsv \
# --output-path paired-end-demux.qza \
# --input-format PairedEndFastqManifestPhred33V2

In [3]:
### Already run -- don't run again if table.qza and rep-seqs.qza already exist
# Sequence quality control using DADA2
# WIP - figure out DADA2 vs Deblur
# Could use FIGARO to dial in truncation params
# !qiime dada2 denoise-paired \
#   --i-demultiplexed-seqs paired-end-demux.qza \
#   --p-trim-left-f 13 \
#   --p-trim-left-r 13 \
#   --p-trunc-len-f 250 \
#   --p-trunc-len-r 250 \
#   --o-table table.qza \
#   --o-representative-sequences rep-seqs.qza \
#   --o-denoising-stats denoising-stats.qza

### Analysis

In [4]:
# Generate a summary of the sequence counts
!qiime demux summarize \
--i-data paired-end-demux.qza \
--o-visualization demux.qzv

Saved Visualization to: demux.qzv


In [5]:
# Show interactive quality plot
# Change tabs to view
Visualization.load('demux.qzv')

<visualization: Visualization uuid: 7c84dd00-ec1e-4985-bd8a-02004530cc96>

In [6]:
# Download a pre-trained Naive Bayes classifier
!wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2023.2/common/gg-13-8-99-515-806-nb-classifier.qza"

--2023-03-15 12:55:15--  https://data.qiime2.org/2023.2/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2023.2/common/gg-13-8-99-515-806-nb-classifier.qza [following]
--2023-03-15 12:55:15--  https://s3-us-west-2.amazonaws.com/qiime2-data/2023.2/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 3.5.80.17, 52.218.237.192, 52.218.245.56, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|3.5.80.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28289645 (27M) [binary/octet-stream]
Saving to: ‘gg-13-8-99-515-806-nb-classifier.qza’

gg-13-8-99-515-806- 100%[===================>]  26.98M  1.52MB/s    in 12s     

2023-03-15 12:55:28 (2.18 MB/s)

In [7]:
# Apply the classifier
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

Saved FeatureData[Taxonomy] to: taxonomy.qza


In [8]:
# Make interactive bar plots for taxonomic composition
!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file ov-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

Saved Visualization to: taxa-bar-plots.qzv


In [11]:
# Show interactive taxonomy bar plot
# Set "Sort Samples By" to "date"
# Set "Taxonomic Level" to 5
Visualization.load('taxa-bar-plots.qzv')

<visualization: Visualization uuid: 9bd2e5c0-3085-43d5-8e36-499caee96416>

### Resources Used
https://docs.qiime2.org/2023.2/tutorials/moving-pictures/

https://john-quensen.com/tutorials/processing-16s-sequences-with-qiime2-and-dada2/